In [ ]:
import soccerdata as sd

# Create scraper class instance
fbref = sd.FBref()

# Create dataframes
season_stats = fbref.read_team_season_stats(stat_type='shooting').head(10)

[07/02/25 10:33:55] INFO     No custom team name replacements found. You can configure these in       ]8;id=573567;file://c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=89491;file://c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py#91\91]8;;\
                             C:\Users\ez\soccerdata\config\teamname_replacements.json.                             

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=101803;file://c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=805821;file://c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py#197\197]8;;\
                             C:\Users\ez\soccerdata\config\league_dict.json.                                       

                    INFO     Saving cached data to C:\Users\ez\soccerdata\data\FBref                 ]8;id=66907;file://c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=602732;file://c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py#263\263]8;;\

                    INFO     No seasons provided. Will retrieve data for the last 5 seasons.         ]8;id=154299;file://c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=486934;file://c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py#475\475]8;;\

                    WARNING  c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccer ]8;id=494804;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=184244;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             data\fbref.py:165: FutureWarning: The behavior of DataFrame                           
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

[07/02/25 10:33:56] WARNING  c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccer ]8;id=417752;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=519223;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             data\_common.py:145: UserWarning: Season id "2021" is ambiguous:                      
                             interpreting as "20-21"                                                               
                               warnings.warn(msg, stacklevel=1)                                                    
                                                                                                                   

In [2]:
season_stats

players_used   90s Standard       \
                                                                 Gls   Sh   
league             season team                                              
ENG-Premier League 1920   Arsenal                  29  38.0       56  403   
                          Aston Villa              28  38.0       40  453   
                          Bournemouth              27  38.0       38  385   
                          Brighton                 25  38.0       35  460   
                          Burnley                  22  38.0       41  384   
                          Chelsea                  27  38.0       69  618   
                          Crystal Palace           25  38.0       29  373   
                          Everton                  24  38.0       42  468   
                          Leicester City           24  38.0       65  533   
                          Liverpool                24  38.0       83  586   

                                                                               \
                                          SoT  SoT%  Sh/90 SoT/90  G/Sh G/SoT   
league             season team                                                  
ENG-Premier League 1920   Arsenal         146  36.2  10.61   3.84  0.13  0.36   
                          Aston Villa     147  32.5  11.92   3.87  0.09  0.27   
                          Bournemouth     117  30.4  10.13   3.08  0.09  0.29   
                          Brighton        139  30.2  12.11   3.66  0.07  0.24   
                          Burnley         124  32.3  10.11   3.26   0.1  0.31   
                          Chelsea         210  34.0  16.26   5.53   0.1   0.3   
                          Crystal Palace  115  30.8   9.82   3.03  0.07  0.23   
                          Everton         159  34.0  12.32   4.18  0.09  0.26   
                          Leicester City  183  34.3  14.03   4.82  0.11  0.33   
                          Liverpool       224  38.2  15.42   5.89  0.13  0.35   

                                                            Expected        \
                                          Dist  FK PK PKatt       xG  npxG   
league             season team                                               
ENG-Premier League 1920   Arsenal         16.4  19  3     3     47.0  44.6   
                          Aston Villa     17.5  21  1     3     44.3  41.8   
                          Bournemouth     17.3  21  4     4     44.8  41.7   
                          Brighton        17.6  11  1     2     45.4  43.8   
                          Burnley         16.4  18  3     3     47.4  45.3   
                          Chelsea         16.6  29  7     7     69.2  64.4   
                          Crystal Palace  17.0  15  3     3     35.8  33.6   
                          Everton         16.4  20  1     1     50.8  50.0   
                          Leicester City  17.4  20  5     7     62.2  56.7   
                          Liverpool       16.5  20  5     5     68.9  65.0   

                                                                \
                                         npxG/Sh  G-xG np:G-xG   
league             season team                                   
ENG-Premier League 1920   Arsenal           0.11   9.0     8.4   
                          Aston Villa       0.09  -4.3    -2.8   
                          Bournemouth       0.11  -6.8    -7.7   
                          Brighton           0.1 -10.4    -9.8   
                          Burnley           0.12  -6.4    -7.3   
                          Chelsea           0.11  -0.2    -2.4   
                          Crystal Palace    0.09  -6.8    -7.6   
                          Everton           0.11  -8.8    -9.0   
                          Leicester City    0.11   2.8     3.3   
                          Liverpool         0.11  14.1    13.0   

                                                                                        url  
                 

In [21]:
elo = sd.ClubElo()
df = elo.read_by_date('2024-10-01')
df.head()

[07/02/25 10:45:16] INFO     Saving cached data to C:\Users\ez\soccerdata\data\ClubElo               ]8;id=832430;file://c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=524712;file://c:\Users\ez\Desktop\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py#263\263]8;;\

,rank,country,level,elo,from,to,league
team,,,,,,,
Man City,1.0,ENG,1,2050.599854,2024-09-29,2024-10-01,ENG-Premier League
Real Madrid,2.0,ESP,1,1997.267700,2024-09-30,2024-10-01,ESP-La Liga
Arsenal,3.0,ENG,1,1964.943115,2024-09-29,2024-10-01,ENG-Premier League
Inter,4.0,ITA,1,1955.128784,2024-09-29,2024-10-01,ITA-Serie A
Leverkusen,5.0,GER,1,1940.878174,2024-09-29,2024-10-01,GER-Bundesliga


In [34]:
import pandas as pd
import soccerdata as sd
from soccerdata import ClubElo
from datetime import timedelta

def collect_understat_data():
    us = sd.Understat(leagues=["ENG-Premier League"], seasons=["2022-23"])
    # 팀별 xG 포함 매치 통계 수집
    df = us.read_team_match_stats()  # 팀 기준 매치 데이터
    df['date'] = pd.to_datetime(df['date']).dt.date
    return df

def collect_elo(start_date, end_date):
    elo = ClubElo()
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    dfs = []

    for d in all_dates:
        try:
            daily_df = elo.read_by_date(d)
            daily_df['date'] = d.date()
            dfs.append(daily_df)
        except Exception:
            continue  # Elo 데이터가 없는 날은 패스

    return pd.concat(dfs, ignore_index=True)

def merge_data(df, elo_df):
    df = df.rename(columns={
        'team': 'h_team', 'opponent': 'a_team',
        'xG_for': 'h_xg', 'xG_against': 'a_xg',
        'goals_for': 'h_goals', 'goals_against': 'a_goals'
    })
    df = df.merge(elo_df.rename(columns={'team':'h_team','elo':'h_elo'}), on=['date','h_team'], how='left')
    df = df.merge(elo_df.rename(columns={'team':'a_team','elo':'a_elo'}), on=['date','a_team'], how='left')
    df['result'] = df.apply(lambda r: 2 if r.h_goals>r.a_goals else (1 if r.h_goals==r.a_goals else 0), axis=1)
    return df


In [ ]:
df = collect_understat_data()
elo_df = collect_elo(df['date'].min(), df['date'].max())
merged = merge_data(df, elo_df)
merged.to_csv('processed/epl_2022.csv', index=False)
print(merged.head())